# Évaluation des Modèles - Challenge Titanic

Ce notebook se concentre sur l'évaluation approfondie des modèles de prédiction pour le challenge Titanic. Nous allons analyser les performances de nos modèles, comprendre leurs forces et faiblesses, et identifier les opportunités d'amélioration.

## Objectifs de l'évaluation:
- Analyser les performances du modèle sur différents segments de données
- Identifier les points forts et les faiblesses des modèles
- Examiner les erreurs de prédiction pour comprendre où le modèle échoue
- Comparer différentes métriques d'évaluation
- Explorer des techniques d'ensemble pour améliorer les performances

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pour l'évaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.metrics import log_loss, brier_score_loss
from sklearn.model_selection import learning_curve, validation_curve

# Pour les techniques d'ensemble
from sklearn.ensemble import VotingClassifier, StackingClassifier

# Pour charger les données et les modèles
from sklearn.externals import joblib
import os
import glob

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set(font_scale=1.2)

# Pour afficher les graphiques dans le notebook
%matplotlib inline

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

## 1. Chargement des données et des modèles

Commençons par charger les données prétraitées et les modèles que nous avons entraînés.

In [ ]:
# Charger les données prétraitées
preprocessed_dir = '/workspaces/titanicML/Data/preprocessed'
preprocessed_file = os.path.join(preprocessed_dir, 'preprocessed_data.pkl')

try:
    # Charger les données prétraitées
    preprocessed_data = joblib.load(preprocessed_file)
    
    # Extraire les ensembles de données
    X_train = preprocessed_data['train_features']
    y_train = preprocessed_data['train_target']
    X_test = preprocessed_data['test_features']
    test_passenger_ids = preprocessed_data['test_passenger_ids']
    feature_names = preprocessed_data['feature_names']
    
    print("Données prétraitées chargées avec succès!")
    print(f"Ensemble d'entraînement: {X_train.shape[0]} observations, {X_train.shape[1]} caractéristiques")
    print(f"Ensemble de test: {X_test.shape[0]} observations, {X_test.shape[1]} caractéristiques")
    
except FileNotFoundError:
    print(f"Erreur: Fichier {preprocessed_file} introuvable.")
    print("Alternative: chargement des fichiers CSV...")
    
    try:
        train_file = os.path.join(preprocessed_dir, 'train_processed.csv')
        test_file = os.path.join(preprocessed_dir, 'test_processed.csv')
        
        train_data = pd.read_csv(train_file)
        X_test = pd.read_csv(test_file)
        
        # Séparer les caractéristiques et la cible dans les données d'entraînement
        y_train = train_data['Survived']
        X_train = train_data.drop('Survived', axis=1)
        
        # Récupérer les IDs des passagers de test depuis le fichier d'origine
        test_data_original = pd.read_csv('/workspaces/titanicML/Data/test.csv')
        test_passenger_ids = test_data_original['PassengerId']
        
        feature_names = X_train.columns.tolist()
        
        print("Données chargées depuis les fichiers CSV.")
        
    except FileNotFoundError:
        print("Erreur: Fichiers CSV également introuvables.")
        print("Veuillez exécuter le notebook de prétraitement des données avant de continuer.")

# Charger les modèles entraînés
models_dir = '/workspaces/titanicML/models'
model_files = glob.glob(os.path.join(models_dir, 'final_model_*.pkl'))

if model_files:
    print(f"\nModèles trouvés: {len(model_files)}")
    
    # Charger chaque modèle
    models = {}
    for model_file in model_files:
        try:
            model_name = os.path.basename(model_file).replace('final_model_', '').replace('.pkl', '').replace('_', ' ')
            model_data = joblib.load(model_file)
            models[model_name] = model_data
            print(f"  - {model_name} chargé avec succès")
        except Exception as e:
            print(f"  - Erreur lors du chargement de {model_file}: {e}")
    
    # Vérifier si au moins un modèle a été chargé
    if models:
        print("\nModèles chargés avec succès!")
    else:
        print("\nAucun modèle n'a pu être chargé.")
else:
    print("\nAucun modèle trouvé dans le dossier des modèles.")
    print("Veuillez exécuter le notebook de modélisation avant de continuer.")

## 2. Division des données pour l'évaluation

Pour une évaluation plus rigoureuse, nous allons diviser les données d'entraînement en ensembles d'entraînement et de validation.

In [ ]:
# Diviser les données pour l'évaluation
from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de validation
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

print(f"Ensemble d'entraînement: {X_train_split.shape[0]} observations")
print(f"Ensemble de validation: {X_val.shape[0]} observations")

# Vérifier la distribution de la variable cible
print("\nDistribution de la variable cible (Survived):")
print(f"Ensemble d'entraînement: {y_train_split.mean()*100:.2f}% de survivants")
print(f"Ensemble de validation: {y_val.mean()*100:.2f}% de survivants")

## 3. Évaluation détaillée des modèles

Analysons en détail les performances de chaque modèle sur l'ensemble de validation.

In [ ]:
# Fonction pour évaluer un modèle en détail
def evaluate_model_in_detail(model, X_train, y_train, X_val, y_val, model_name):
    # S'assurer que le modèle est entraîné sur les données d'entraînement
    if hasattr(model, 'fit'):
        model.fit(X_train, y_train)
    
    # Faire des prédictions
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Métriques de base
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    # Métriques avancées
    if y_pred_proba is not None:
        auc = roc_auc_score(y_val, y_pred_proba)
        avg_precision = average_precision_score(y_val, y_pred_proba)
        log_loss_val = log_loss(y_val, y_pred_proba)
        brier = brier_score_loss(y_val, y_pred_proba)
    else:
        auc = None
        avg_precision = None
        log_loss_val = None
        brier = None
    
    # Matrice de confusion
    cm = confusion_matrix(y_val, y_pred)
    
    # Afficher les résultats
    print(f"\n=== Évaluation détaillée de {model_name} ===")
    print("\nMétriques de classification:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-score: {f1:.4f}")
    
    if auc is not None:
        print("\nMétriques de probabilité:")
        print(f"  AUC-ROC: {auc:.4f}")
        print(f"  Précision moyenne: {avg_precision:.4f}")
        print(f"  Log Loss: {log_loss_val:.4f}")
        print(f"  Brier Score: {brier:.4f}")
    
    # Afficher la matrice de confusion
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=['Prédit: Non-survie', 'Prédit: Survie'],
               yticklabels=['Réel: Non-survie', 'Réel: Survie'])
    plt.title(f'Matrice de confusion - {model_name}', fontsize=16)
    plt.ylabel('Valeur réelle', fontsize=12)
    plt.xlabel('Valeur prédite', fontsize=12)
    plt.show()
    
    # Afficher le rapport de classification
    print("\nRapport de classification:")
    print(classification_report(y_val, y_pred, target_names=['Non-survie', 'Survie']))
    
    # Tracer la courbe ROC si disponible
    if y_pred_proba is not None:
        plt.figure(figsize=(8, 6))
        fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
        plt.plot(fpr, tpr, label=f'AUC = {auc:.4f}')
        plt.plot([0, 1], [0, 1], 'k--', label='Aléatoire')
        plt.xlabel('Taux de faux positifs', fontsize=12)
        plt.ylabel('Taux de vrais positifs', fontsize=12)
        plt.title(f'Courbe ROC - {model_name}', fontsize=16)
        plt.legend(loc='lower right')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Tracer la courbe Precision-Recall
        plt.figure(figsize=(8, 6))
        precision_vals, recall_vals, _ = precision_recall_curve(y_val, y_pred_proba)
        plt.plot(recall_vals, precision_vals, label=f'AP = {avg_precision:.4f}')
        plt.axhline(y=y_val.mean(), color='r', linestyle='--', label='Aléatoire')
        plt.xlabel('Recall', fontsize=12)
        plt.ylabel('Precision', fontsize=12)
        plt.title(f'Courbe Precision-Recall - {model_name}', fontsize=16)
        plt.legend(loc='upper right')
        plt.grid(True, alpha=0.3)
        plt.show()
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'avg_precision': avg_precision,
        'log_loss': log_loss_val,
        'brier_score': brier,
        'confusion_matrix': cm,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

# Évaluer chaque modèle chargé
evaluation_results = {}

for model_name, model_data in models.items():
    model = model_data['model']
    results = evaluate_model_in_detail(model, X_train_split, y_train_split, X_val, y_val, model_name)
    evaluation_results[model_name] = results

# Comparer les modèles sur les métriques principales
metrics_df = pd.DataFrame({
    'Modèle': list(evaluation_results.keys()),
    'Accuracy': [results['accuracy'] for results in evaluation_results.values()],
    'Precision': [results['precision'] for results in evaluation_results.values()],
    'Recall': [results['recall'] for results in evaluation_results.values()],
    'F1-score': [results['f1'] for results in evaluation_results.values()]
})

# Ajouter les métriques de probabilité si disponibles
if all(results['auc'] is not None for results in evaluation_results.values()):
    metrics_df['AUC-ROC'] = [results['auc'] for results in evaluation_results.values()]
    metrics_df['Précision moyenne'] = [results['avg_precision'] for results in evaluation_results.values()]
    metrics_df['Log Loss'] = [results['log_loss'] for results in evaluation_results.values()]
    metrics_df['Brier Score'] = [results['brier_score'] for results in evaluation_results.values()]

# Afficher le tableau comparatif
display(metrics_df.sort_values('F1-score', ascending=False))

# Visualiser la comparaison des métriques principales
plt.figure(figsize=(12, 8))
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-score']
metrics_df_melted = pd.melt(metrics_df, id_vars=['Modèle'], value_vars=metrics_to_plot,
                          var_name='Métrique', value_name='Valeur')

sns.barplot(x='Métrique', y='Valeur', hue='Modèle', data=metrics_df_melted)
plt.title('Comparaison des performances des modèles', fontsize=16)
plt.ylabel('Score', fontsize=12)
plt.ylim(0.7, 1.0)  # Ajuster selon vos résultats
plt.legend(title='Modèle')
plt.tight_layout()
plt.show()

## 4. Analyse des erreurs de prédiction

Examinons en détail où et pourquoi nos modèles font des erreurs de prédiction.

In [ ]:
# Sélectionner le meilleur modèle pour l'analyse des erreurs
if evaluation_results:
    best_model_name = metrics_df.sort_values('F1-score', ascending=False).iloc[0]['Modèle']
    best_model_results = evaluation_results[best_model_name]
    best_model = models[best_model_name]['model']
    print(f"Analyse des erreurs pour le meilleur modèle: {best_model_name}")
else:
    print("Aucun modèle disponible pour l'analyse des erreurs.")
    # Si aucun modèle n'est disponible, utiliser un modèle de secours
    from sklearn.ensemble import RandomForestClassifier
    best_model = RandomForestClassifier(random_state=42)
    best_model.fit(X_train_split, y_train_split)
    y_pred = best_model.predict(X_val)
    y_pred_proba = best_model.predict_proba(X_val)[:, 1]
    best_model_name = "Random Forest (secours)"

# Fonction pour analyser les erreurs
def analyze_prediction_errors(model, X_val, y_val, feature_names):
    # Prédictions
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Créer un DataFrame pour l'analyse
    val_data = X_val.copy()
    val_data['Survived_Actual'] = y_val
    val_data['Survived_Predicted'] = y_pred
    if y_pred_proba is not None:
        val_data['Survival_Probability'] = y_pred_proba
    
    # Identifier les erreurs
    val_data['Correct_Prediction'] = (val_data['Survived_Actual'] == val_data['Survived_Predicted']).astype(int)
    val_data['Error_Type'] = 'Correct'
    val_data.loc[(val_data['Survived_Actual'] == 1) & (val_data['Survived_Predicted'] == 0), 'Error_Type'] = 'Faux Négatif'
    val_data.loc[(val_data['Survived_Actual'] == 0) & (val_data['Survived_Predicted'] == 1), 'Error_Type'] = 'Faux Positif'
    
    # Afficher la distribution des erreurs
    error_counts = val_data['Error_Type'].value_counts()
    print("\nDistribution des prédictions:")
    print(error_counts)
    
    # Visualiser la distribution des erreurs
    plt.figure(figsize=(10, 6))
    sns.countplot(x='Error_Type', data=val_data, palette={'Correct': '#4CAF50', 'Faux Négatif': '#FF9800', 'Faux Positif': '#F44336'})
    plt.title('Distribution des erreurs de prédiction', fontsize=16)
    plt.xlabel('Type d\'erreur', fontsize=12)
    plt.ylabel('Nombre d\'observations', fontsize=12)
    
    # Ajouter les pourcentages sur les barres
    for i, count in enumerate(error_counts):
        percentage = count / len(val_data) * 100
        plt.text(i, count + 5, f'{count} ({percentage:.1f}%)', ha='center')
    
    plt.show()
    
    # Analyser les erreurs par caractéristiques clés
    if 'Sex_Code' in val_data.columns:
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        sns.countplot(x='Sex_Code', hue='Error_Type', data=val_data, 
                     palette={'Correct': '#4CAF50', 'Faux Négatif': '#FF9800', 'Faux Positif': '#F44336'})
        plt.title('Erreurs par sexe')
        plt.xlabel('Sexe (0 = Homme, 1 = Femme)')
        plt.ylabel('Nombre d\'observations')
        
        plt.subplot(1, 2, 2)
        sex_error_rate = val_data.groupby('Sex_Code')['Correct_Prediction'].mean() * 100
        sns.barplot(x=sex_error_rate.index, y=sex_error_rate.values)
        plt.title('Taux de prédiction correcte par sexe')
        plt.xlabel('Sexe (0 = Homme, 1 = Femme)')
        plt.ylabel('Taux de prédiction correcte (%)')
        plt.tight_layout()
        plt.show()
    
    # Analyse par classe
    if 'Pclass' in val_data.columns:
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        sns.countplot(x='Pclass', hue='Error_Type', data=val_data,
                     palette={'Correct': '#4CAF50', 'Faux Négatif': '#FF9800', 'Faux Positif': '#F44336'})
        plt.title('Erreurs par classe')
        plt.xlabel('Classe')
        plt.ylabel('Nombre d\'observations')
        
        plt.subplot(1, 2, 2)
        class_error_rate = val_data.groupby('Pclass')['Correct_Prediction'].mean() * 100
        sns.barplot(x=class_error_rate.index, y=class_error_rate.values)
        plt.title('Taux de prédiction correcte par classe')
        plt.xlabel('Classe')
        plt.ylabel('Taux de prédiction correcte (%)')
        plt.tight_layout()
        plt.show()
    
    # Analyse par âge (si disponible)
    if 'Age' in val_data.columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='Error_Type', y='Age', data=val_data,
                   palette={'Correct': '#4CAF50', 'Faux Négatif': '#FF9800', 'Faux Positif': '#F44336'})
        plt.title('Distribution des erreurs par âge', fontsize=16)
        plt.xlabel('Type d\'erreur', fontsize=12)
        plt.ylabel('Âge', fontsize=12)
        plt.show()
    
    # Analyse des cas difficiles (probabilités proches de 0.5)
    if 'Survival_Probability' in val_data.columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(data=val_data, x='Survival_Probability', hue='Error_Type', bins=30, 
                    palette={'Correct': '#4CAF50', 'Faux Négatif': '#FF9800', 'Faux Positif': '#F44336'})
        plt.title('Distribution des probabilités par type d\'erreur', fontsize=16)
        plt.xlabel('Probabilité prédite de survie', fontsize=12)
        plt.ylabel('Nombre d\'observations', fontsize=12)
        plt.axvline(x=0.5, color='black', linestyle='--', label='Seuil de décision')
        plt.legend()
        plt.show()
    
    # Identifier les cas les plus difficiles
    if 'Survival_Probability' in val_data.columns:
        # Calculer la distance au seuil de décision (0.5)
        val_data['Threshold_Distance'] = abs(val_data['Survival_Probability'] - 0.5)
        
        # Filtrer les prédictions incorrectes
        incorrect_predictions = val_data[val_data['Correct_Prediction'] == 0]
        
        # Trier par proximité au seuil
        difficult_cases = incorrect_predictions.sort_values('Threshold_Distance').head(10)
        
        print("\nLes 10 cas les plus difficiles (prédictions incorrectes proches du seuil):")
        display(difficult_cases[['Survived_Actual', 'Survived_Predicted', 'Survival_Probability', 'Error_Type', 'Threshold_Distance'] + 
                             [col for col in val_data.columns if col in ['Pclass', 'Sex_Code', 'Age', 'Fare', 'FamilySize']]])
    
    return val_data

# Analyser les erreurs du meilleur modèle
error_analysis_data = analyze_prediction_errors(best_model, X_val, y_val, feature_names)

## 5. Courbes d'apprentissage

Analysons comment les performances du modèle évoluent en fonction de la taille de l'ensemble d'entraînement pour détecter d'éventuels problèmes de sur-ajustement ou de sous-ajustement.

In [ ]:
# Courbes d'apprentissage pour le meilleur modèle
def plot_learning_curve(estimator, X, y, title="Courbe d'apprentissage", ylim=None, cv=5,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 10)):
    plt.figure(figsize=(10, 6))
    plt.title(title, fontsize=16)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Taille de l'ensemble d'entraînement", fontsize=12)
    plt.ylabel("Score", fontsize=12)
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='accuracy')
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    # Tracer le score sur l'ensemble d'entraînement
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="#ff9800")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="#ff9800",
             label="Score d'entraînement")
    
    # Tracer le score sur l'ensemble de validation
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="#2196f3")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="#2196f3",
             label="Score de validation")
    
    plt.grid(True, alpha=0.3)
    plt.legend(loc="best")
    
    # Ajouter des lignes horizontales pour le score de référence
    plt.axhline(y=0.5, color='r', linestyle='--', label="Baseline (50%)")
    plt.axhline(y=y.mean(), color='g', linestyle='--', label=f"Baseline (taux de survie: {y.mean():.2f})")
    
    plt.legend()
    plt.show()
    
    return train_scores_mean, test_scores_mean

# Tracer la courbe d'apprentissage pour le meilleur modèle
print(f"\nCourbe d'apprentissage pour {best_model_name}:")
train_scores, test_scores = plot_learning_curve(
    best_model, X_train, y_train, 
    title=f"Courbe d'apprentissage - {best_model_name}", 
    cv=5, n_jobs=-1
)

## 6. Techniques d'ensemble pour améliorer les performances

Explorons les techniques d'ensemble pour combiner les prédictions de différents modèles et potentiellement améliorer les performances.

In [ ]:
# Vérifier si nous avons plusieurs modèles pour créer un ensemble
if len(models) >= 2:
    print("Création de modèles d'ensemble à partir des modèles disponibles...")
    
    # Créer une liste des modèles à utiliser
    ensemble_models = [(name, model_data['model']) for name, model_data in models.items()]
    
    # Créer un modèle de vote (soft voting)
    voting_classifier = VotingClassifier(
        estimators=ensemble_models,
        voting='soft'  # Utiliser le vote pondéré par les probabilités
    )
    
    # Entraîner le modèle de vote
    voting_classifier.fit(X_train_split, y_train_split)
    
    # Évaluer le modèle de vote
    voting_results = evaluate_model_in_detail(
        voting_classifier, X_train_split, y_train_split, X_val, y_val, "Voting Classifier (soft)"
    )
    
    # Créer un modèle de stacking
    stacking_classifier = StackingClassifier(
        estimators=ensemble_models,
        final_estimator=LogisticRegression(),
        cv=5
    )
    
    # Entraîner le modèle de stacking
    stacking_classifier.fit(X_train_split, y_train_split)
    
    # Évaluer le modèle de stacking
    stacking_results = evaluate_model_in_detail(
        stacking_classifier, X_train_split, y_train_split, X_val, y_val, "Stacking Classifier"
    )
    
    # Comparer les performances
    ensemble_comparison = pd.DataFrame({
        'Modèle': [best_model_name, "Voting Classifier", "Stacking Classifier"],
        'Accuracy': [best_model_results['accuracy'], voting_results['accuracy'], stacking_results['accuracy']],
        'F1-score': [best_model_results['f1'], voting_results['f1'], stacking_results['f1']],
        'AUC-ROC': [best_model_results['auc'], voting_results['auc'], stacking_results['auc']]
    })
    
    print("\nComparaison des performances des modèles d'ensemble:")
    display(ensemble_comparison.sort_values('F1-score', ascending=False))
    
    # Si le modèle d'ensemble est meilleur, l'utiliser pour les prédictions finales
    best_ensemble_model = None
    if ensemble_comparison.iloc[0]['Modèle'] == "Voting Classifier":
        best_ensemble_model = voting_classifier
        print("\nLe modèle de vote (Voting Classifier) est le plus performant!")
    elif ensemble_comparison.iloc[0]['Modèle'] == "Stacking Classifier":
        best_ensemble_model = stacking_classifier
        print("\nLe modèle de stacking (Stacking Classifier) est le plus performant!")
    
    # Si un modèle d'ensemble est meilleur, l'utiliser pour les prédictions finales
    if best_ensemble_model is not None and ensemble_comparison.iloc[0]['F1-score'] > best_model_results['f1']:
        print("Utilisation du modèle d'ensemble pour les prédictions finales...")
        
        # Faire des prédictions sur l'ensemble de test
        ensemble_test_predictions = best_ensemble_model.predict(X_test)
        
        # Créer le DataFrame pour la soumission
        ensemble_submission = pd.DataFrame({
            'PassengerId': test_passenger_ids,
            'Survived': ensemble_test_predictions
        })
        
        # Sauvegarder le fichier de soumission
        submissions_dir = '/workspaces/titanicML/submissions'
        if not os.path.exists(submissions_dir):
            os.makedirs(submissions_dir)
        
        ensemble_submission_file = os.path.join(submissions_dir, 'submission_ensemble_model.csv')
        ensemble_submission.to_csv(ensemble_submission_file, index=False)
        
        print(f"Fichier de soumission du modèle d'ensemble créé: {ensemble_submission_file}")
        
        # Sauvegarder le modèle d'ensemble
        ensemble_model_data = {
            'model': best_ensemble_model,
            'model_name': ensemble_comparison.iloc[0]['Modèle'],
            'feature_names': feature_names,
            'base_models': ensemble_models,
            'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        ensemble_model_file = os.path.join(models_dir, 'final_model_ensemble.pkl')
        joblib.dump(ensemble_model_data, ensemble_model_file)
        
        print(f"Modèle d'ensemble sauvegardé: {ensemble_model_file}")
else:
    print("Pas assez de modèles disponibles pour créer un ensemble.")

## 7. Conclusions et recommandations

Résumons les résultats de notre évaluation et proposons des recommandations pour améliorer les performances des modèles.

In [ ]:
# Collecter les informations clés pour le résumé
if evaluation_results:
    # Trouver le meilleur modèle
    best_model_row = metrics_df.sort_values('F1-score', ascending=False).iloc[0]
    best_model_name = best_model_row['Modèle']
    best_f1 = best_model_row['F1-score']
    best_accuracy = best_model_row['Accuracy']
    
    # Caractéristiques importantes (si disponibles)
    feature_importance = None
    if hasattr(models[best_model_name]['model'], 'feature_importances_'):
        importances = models[best_model_name]['model'].feature_importances_
        feature_importance = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values('Importance', ascending=False).head(10)
    elif hasattr(models[best_model_name]['model'], 'coef_'):
        importances = np.abs(models[best_model_name]['model'].coef_[0])
        feature_importance = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values('Importance', ascending=False).head(10)
    
    # Analyser les erreurs fréquentes
    if 'Error_Type' in error_analysis_data.columns:
        error_counts = error_analysis_data['Error_Type'].value_counts()
        error_rate = (error_counts['Faux Négatif'] + error_counts['Faux Positif']) / len(error_analysis_data)
        fn_rate = error_counts['Faux Négatif'] / (error_counts['Faux Négatif'] + error_counts.get('Correct', 0))
        fp_rate = error_counts['Faux Positif'] / (error_counts['Faux Positif'] + error_counts.get('Correct', 0))
    
    # Afficher le résumé
    print("\n=== RÉSUMÉ DE L'ÉVALUATION DES MODÈLES ===")
    print(f"\n1. Meilleur modèle: {best_model_name}")
    print(f"   - Accuracy: {best_accuracy:.4f}")
    print(f"   - F1-score: {best_f1:.4f}")
    
    if feature_importance is not None:
        print("\n2. Caractéristiques les plus importantes:")
        for i, (feature, importance) in enumerate(zip(feature_importance['Feature'], feature_importance['Importance']), 1):
            print(f"   {i}. {feature}: {importance:.4f}")
    
    print("\n3. Analyse des erreurs:")
    print(f"   - Taux d'erreur global: {error_rate:.4f} ({error_rate*100:.1f}%)")
    print(f"   - Taux de faux négatifs (survies manquées): {fn_rate:.4f} ({fn_rate*100:.1f}%)")
    print(f"   - Taux de faux positifs (décès manqués): {fp_rate:.4f} ({fp_rate*100:.1f}%)")
    
    # Tracer la matrice de confusion pour le résumé
    plt.figure(figsize=(8, 6))
    cm = best_model_results['confusion_matrix']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=['Prédit: Non-survie', 'Prédit: Survie'],
               yticklabels=['Réel: Non-survie', 'Réel: Survie'])
    plt.title(f'Matrice de confusion - {best_model_name}', fontsize=16)
    plt.ylabel('Valeur réelle', fontsize=12)
    plt.xlabel('Valeur prédite', fontsize=12)
    plt.show()
    
    # Recommandations
    print("\n=== RECOMMANDATIONS POUR AMÉLIORER LES PERFORMANCES ===")
    print("\n1. Collecte de données supplémentaires:")
    print("   - Rechercher des informations sur la disposition des cabines et leur proximité avec les canots de sauvetage")
    print("   - Intégrer des données sur les liens familiaux entre passagers")
    
    print("\n2. Ingénierie de caractéristiques:")
    print("   - Créer des caractéristiques plus granulaires pour l'âge et les tarifs")
    print("   - Explorer des interactions entre variables (ex: âge × classe)")
    print("   - Extraire plus d'informations des noms (nationalité, etc.)")
    
    print("\n3. Optimisation des modèles:")
    print("   - Tester des techniques d'ensemble plus avancées")
    print("   - Optimiser les seuils de décision pour réduire les faux négatifs ou les faux positifs selon l'objectif")
    print("   - Explorer des réseaux de neurones profonds avec une architecture adaptée")
    
    print("\n4. Validation et généralisation:")
    print("   - Effectuer une validation croisée plus robuste avec plus de folds")
    print("   - Tester la robustesse du modèle sur différents échantillons des données")

## 8. Prochaines étapes

Pour finaliser notre projet sur le challenge Titanic, nous devrons:

1. **Sélectionner le modèle final** en fonction des résultats de cette évaluation approfondie

2. **Générer les prédictions finales** pour l'ensemble de test avec le meilleur modèle (individuel ou d'ensemble)

3. **Préparer le fichier de soumission** au format requis par Kaggle

4. **Soumettre nos prédictions** sur la plateforme Kaggle et évaluer notre score

5. **Documenter le processus complet** de notre approche, y compris:
   - Prétraitement des données
   - Création de caractéristiques
   - Sélection et optimisation des modèles
   - Résultats d'évaluation
   - Leçons apprises et améliorations potentielles

La dernière étape de notre projet consistera à créer le fichier de soumission final et à le soumettre à Kaggle.